In [1]:
%load_ext autoreload
%autoreload 2
# %cd /p/mmcardiac/Jerry/code/CINE-DENSE-Translation
%cd /home/jx8fh/Projects/LaMoD
# %cd /workspace/LaMoD
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from modules.data import check_dict

/sfs/gpfs/tardis/home/jx8fh/Projects/LaMoD


In [2]:
import json
config_fname = 'configs/ours-LaMoD.json'
config = json.load(open(config_fname))

In [3]:
cine_DENSE_fname = '/scratch/jx8fh/2024-04-21-matched-cine-DENSE-from-Yu-all-with-resampled.npy'
cine_DENSE_data = np.load(cine_DENSE_fname, allow_pickle=True).tolist()

In [4]:
print(f"length of data: {len(cine_DENSE_data)}")
check_dict(cine_DENSE_data[0])

length of data: 105
AnalysisInfo                                                 dict_keys(['ResampleMethod', 'ResampleDistance', 'SpatialSmoothing', 'TemporalOrder', 'Xseed', 'Yseed', 'Zseed', 'FramesForAnalysis', 'Nmodel', 'PositionA', 'PositionB', 'Clockwise'])
AnalysisInstanceUID                                          1.3.6.1.4.1.9590.100.1.2.64946301013757307506911904523491151334
DENSEInfo                                                    dict_keys(['Name', 'UID', 'Type', 'MagIndex', 'PhaIndex', 'Number', 'PixelSpacing', 'Scale', 'EncFreq', 'SwapFlag', 'NegFlag'])
DENSE_Eul_displacement_field_Multipliers                     (3,)                
DENSE_Eul_displacement_field_X                               (48, 48, 51)        
DENSE_Eul_displacement_field_Y                               (48, 48, 51)        
DENSE_Lag_displacement_field_X                               (48, 48, 51)        
DENSE_Lag_displacement_field_Y                               (48, 48, 51)        
DENSE_frame

In [5]:
from modules.data.processing.strain_analysis.pixelstrain import pixelstrain
from modules.data.processing.strain_analysis.resample_strain_matrix import resample_strain_matrix_2d
from modules.data.processing.strain_analysis.six_segments_core import six_segments_core, get_various_segmental_strain_data
import numpy as np
from pathlib import Path
import tqdm
from copy import deepcopy
import multiprocessing

def process_repeat(repeat_data):
    H, W = repeat_data['disp_pred'].shape[1:3]
    X, Y = np.meshgrid(np.arange(W), np.arange(H))
    dXt = repeat_data['disp_pred_unmasked'][0]
    dYt = repeat_data['disp_pred_unmasked'][1]
    mask = repeat_data['mask']
    Nfr = repeat_data['disp_pred'].shape[-1]
    
    strain = pixelstrain(X=X, Y=Y, dXt=dXt, dYt=dYt, mask=mask, times=np.arange(Nfr))
    strainCC = strain['CC']

    mask_GT = (np.abs(repeat_data['disp_GT'][0, ..., 0]) + np.abs(repeat_data['disp_GT'][1, ..., 0])) > 0
    Nfr_DENSE = repeat_data['disp_GT'].shape[-1]
    cine_strain_GT = pixelstrain(
        X=X, Y=Y, 
        dXt=repeat_data['disp_GT'][0], 
        dYt=repeat_data['disp_GT'][1], 
        mask=mask_GT, times=np.arange(Nfr_DENSE))
    
    cine_recompute_strain = get_various_segmental_strain_data(
        strain_images=strain['CC'], 
        strain_mask=~np.isnan(strain['CC'][...,0]), 
        origin=np.array([24,24]), 
        insertion=repeat_data['cine_rv_insertion_points_rescaled_DENSE48'],
        enable_offset=True,
        offset_degrees=[90, 120, 120],
        generate_layerid_map=False)

    cine_DENSE_strain_imgs = cine_strain_GT['CC']
    cine_DENSE_strain_mask = np.abs(repeat_data['disp_GT'][0, ..., 0]) > 1e-5
    cine_DENSE_origin = repeat_data['AnalysisInfo']['PositionA'] - np.array([repeat_data['DENSE_myo_mask_bbox'][2], repeat_data['DENSE_myo_mask_bbox'][0]])
    cine_DENSE_insertion = repeat_data['AnalysisInfo']['PositionB'] - np.array([repeat_data['DENSE_myo_mask_bbox'][2], repeat_data['DENSE_myo_mask_bbox'][0]])
    cine_DENSE_recompute_strain = get_various_segmental_strain_data(
        cine_DENSE_strain_imgs, 
        cine_DENSE_strain_mask, 
        cine_DENSE_origin, 
        cine_DENSE_insertion,
        layerid_map = None,
        generate_layerid_map=True)
    
    cine_DENSE_time_stamps = repeat_data['DENSE_frame_time_stamps']
    cine_time_stamps = repeat_data['cine_frame_time_stamps']
    
    for strainmat_name in ['CCs4', 'CCs6', 'CCs18']:
        cine_DENSE_recompute_strain[strainmat_name+'Resampled'] = resample_strain_matrix_2d(cine_DENSE_recompute_strain[strainmat_name], cine_DENSE_time_stamps, cine_time_stamps)
        cine_recompute_strain[strainmat_name+'Resampled'] = resample_strain_matrix_2d(cine_recompute_strain[strainmat_name], cine_time_stamps, cine_DENSE_time_stamps)

    strain_result = {
        'strain_GT': cine_DENSE_strain_imgs,
        'strain_pred': strain['CC']
    }
    for strainmat_name in cine_recompute_strain.keys():
        strain_result[strainmat_name+'_pred'] = cine_recompute_strain[strainmat_name]
    for strainmat_name in cine_DENSE_recompute_strain.keys():
        strain_result[strainmat_name+'_GT'] = cine_DENSE_recompute_strain[strainmat_name]
    
    return strain_result

cine_save_pred_dir = Path('/scratch/jx8fh/2025-01-16-LaMoD-variability-split-100/cine')
cine_save_strain_dir = Path('/scratch/jx8fh/2025-01-16-LaMoD-variability-split-100/cine_strain')
cine_save_strain_dir.mkdir(exist_ok=True, parents=True)

for file_path in tqdm.tqdm(list(cine_save_pred_dir.glob('*.npy'))):
    patient_repeats = np.load(file_path, allow_pickle=True)
    
    # Use multiprocessing to parallelize the strain computation
    with multiprocessing.Pool() as pool:
        strain_results = pool.map(process_repeat, patient_repeats)
    
    strain_save_path = cine_save_strain_dir / file_path.name
    np.save(strain_save_path, strain_results)

  1%|          | 1/105 [00:11<19:13, 11.09s/it]/sfs/gpfs/tardis/home/jx8fh/Projects/LaMoD/modules/data/processing/strain_analysis/six_segments_core.py:266: RuntimeWarning: Mean of empty slice.
  curr_CC_of_seg = strain_images[curr_segmental_layer_mask, :].mean(axis=0)
/sfs/gpfs/tardis/home/jx8fh/Projects/LaMoD/modules/data/processing/strain_analysis/six_segments_core.py:266: RuntimeWarning: Mean of empty slice.
  curr_CC_of_seg = strain_images[curr_segmental_layer_mask, :].mean(axis=0)
/sfs/gpfs/tardis/home/jx8fh/Projects/LaMoD/modules/data/processing/strain_analysis/six_segments_core.py:266: RuntimeWarning: Mean of empty slice.
  curr_CC_of_seg = strain_images[curr_segmental_layer_mask, :].mean(axis=0)
/sfs/gpfs/tardis/home/jx8fh/Projects/LaMoD/modules/data/processing/strain_analysis/six_segments_core.py:266: RuntimeWarning: Mean of empty slice.
  curr_CC_of_seg = strain_images[curr_segmental_layer_mask, :].mean(axis=0)
/sfs/gpfs/tardis/home/jx8fh/Projects/LaMoD/modules/data/processin